# Transformer 구현하기

## 필요한 라이브러리 import하기

In [ ]:
from google.comlab import drive
drive.mount('/content/drive')
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts